# Примеры заполнения таблицы

## Заполенение средствами clickhouse_connect

Способ используется при небольших объемах данных. 

In [ ]:
import clickhouse_connect
import os

In [ ]:
# подключение к системной базе данных
HOST=os.environ['CLICKHOUSE_HOST']
PORT=os.environ['CLICKHOUSE_PORT']
DATABASE=os.environ['CLICKHOUSE_DATABASE']
LOGIN=os.environ['CLICKHOUSE_LOGIN']
PASSWORD=os.environ['CLICKHOUSE_PASSWORD']

In [ ]:
connection = clickhouse_connect.get_client(host=HOST, 
                                    port=int(PORT), 
                                    database = DATABASE,
                                    username=LOGIN, 
                                    password=PASSWORD,
                                    send_receive_timeout=12000,
                                    query_limit=0)    

In [ ]:
# имя таблицы созданной в предыдущих примерах
tableName = 'table1'

In [ ]:
sql = f"""
    insert into {DATABASE}.{tableName} (c1, c2, c3) values (1,1.0,'1')
"""
print(sql)
result = connection.command(sql)   

In [ ]:
connection.close()

In [ ]:
# проверка записанных данных
connection = clickhouse_connect.get_client(host=HOST, 
                                    port=int(PORT), 
                                    database = DATABASE,
                                    username=LOGIN, 
                                    password=PASSWORD,
                                    send_receive_timeout=12000,
                                    query_limit=0) 
sql = f"""
     select * FROM {DATABASE}.{tableName}
"""
print(sql)
result = connection.command(sql)
print(result)
connection.close()

## Заполенение средствами spark

Используется на больших объемах данных.

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit


In [ ]:
# запуск spark
spark = SparkSession.builder.appName('fill').getOrCreate()

In [ ]:
jdbcURLClickhouse = f'jdbc:clickhouse://{HOST}:{PORT}/{DATABASE}'

In [ ]:
# формирование dataframe
values= [
        (4,4.0,'4'),
        (5,5.0,'5'),
        (6,6.0,'6')
    ]

schemaData = StructType([ \
    StructField("c1",IntegerType(),True), 
    StructField("c2",FloatType(),True), 
    StructField("c3",StringType(),True)
  ])

df = spark.createDataFrame(values,schema=schemaData)
df.show(3,False)

In [ ]:
# запись в таблицу
df.write.format("jdbc").mode("append") \
              .option('url', jdbcURLClickhouse) \
              .option('driver', 'com.clickhouse.jdbc.ClickHouseDriver') \
              .option('user',LOGIN) \
              .option('password',PASSWORD) \
              .option("dbtable",DATABASE+'.'+tableName).save()

In [ ]:
# чтение из таблицы
query=f'''
SELECT  *
FROM {DATABASE}.{tableName}
'''
df1 = spark.read.format("jdbc") \
              .option('url', jdbcURLClickhouse) \
              .option('driver', 'com.clickhouse.jdbc.ClickHouseDriver') \
              .option('user',LOGIN) \
              .option('password',PASSWORD) \
              .option('query',query).load()
df1.show()

In [ ]:
spark.stop()